<a href="https://colab.research.google.com/github/Pmilivojevic/PyTorch/blob/main/LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
num_channels = 3
num_classes = 10
batch_size = 1024
lr = 0.001
num_epochs = 50

In [ ]:
train_dataset = datasets.CIFAR10(
    root='/content/drive/MyDrive/Colab Notebooks/Dataset',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.CIFAR10(
    root='/content/drive/MyDrive/Colab Notebooks/Dataset',
    train=False,
    transform=transforms.ToTensor(),
    download=True
)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class LeNet(nn.Module):
  def __init__(self, num_channels: int=3, num_classes: int=10):
    super(LeNet, self).__init__()
    self.num_channels = num_channels
    self.num_classes = num_classes
    self.relu = nn.ReLU()
    self.pool = nn.AvgPool2d(kernel_size=(2,2), stride=(2,2))
    self.conv1 = nn.Conv2d(in_channels=num_channels, out_channels=6, kernel_size=(5,5), padding=(2,2))
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5,5), padding=(2,2))
    self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=(5,5), padding=(1,1))
    self.conv4 = nn.Conv2d(in_channels=120, out_channels=120, kernel_size=(3,3), padding=(0,0))
    self.fc1 = nn.Linear(in_features=120, out_features=84)
    self.fc2 = nn.Linear(in_features=84, out_features=num_classes)

  def forward(self, x):
    x = self.relu(self.conv1(x))
    x = self.pool(x)
    x = self.relu(self.conv2(x))
    x = self.pool(x)
    x = self.relu(self.conv3(x))
    x = self.pool(x)
    x = self.relu(self.conv4(x))
    x = x.reshape(x.shape[0], -1)
    x = self.relu(self.fc1(x))
    x = self.fc2(x)

    return x

In [ ]:
model = LeNet().to(device)

In [ ]:
def check_accuracy(loader, model):
  # if loader.dataset.train:
  #   print('Training data')
  # else:
  #   print('Test data')

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device).squeeze(1)
      y = y.to(device)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    # print(f'{float(num_correct)/float(num_samples)*100:.2f}')

  model.train()

  return float(num_correct)/float(num_samples)*100

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(num_epochs):
  if epoch > 20:
    lr = 0.0001
    optimizer = optim.Adam(model.parameters(), lr=lr)

  for batch_ind, (data, targets) in enumerate(train_loader):
    data = data.to(device).squeeze(1)
    targets = targets.to(device)

    scores = model(data)
    loss = criterion(scores, targets)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()
  print(f'{epoch} Train acc: {check_accuracy(train_loader, model):.2f}, Validation acc: {check_accuracy(test_loader, model):.2f}')

0 Trayn acc: 39.91, Validation acc: 40.12
1 Trayn acc: 42.16, Validation acc: 42.42
2 Trayn acc: 43.23, Validation acc: 43.86
3 Trayn acc: 44.69, Validation acc: 45.05
4 Trayn acc: 46.23, Validation acc: 46.36
5 Trayn acc: 47.22, Validation acc: 47.27
6 Trayn acc: 47.16, Validation acc: 47.46
7 Trayn acc: 48.85, Validation acc: 48.93
8 Trayn acc: 50.47, Validation acc: 50.26
9 Trayn acc: 49.84, Validation acc: 49.63
10 Trayn acc: 52.19, Validation acc: 51.50
11 Trayn acc: 52.53, Validation acc: 52.10
12 Trayn acc: 52.75, Validation acc: 52.13
13 Trayn acc: 53.38, Validation acc: 52.42
14 Trayn acc: 54.18, Validation acc: 53.16
15 Trayn acc: 55.93, Validation acc: 54.43
16 Trayn acc: 55.68, Validation acc: 54.52
17 Trayn acc: 56.49, Validation acc: 55.33
18 Trayn acc: 57.35, Validation acc: 55.63
19 Trayn acc: 57.02, Validation acc: 55.32
20 Trayn acc: 59.06, Validation acc: 56.95
21 Trayn acc: 59.51, Validation acc: 57.35
22 Trayn acc: 59.73, Validation acc: 57.65
23 Trayn acc: 59.83, 

In [ ]:
def check_accuracy(loader, model):
  # if loader.dataset.train:
  #   print('Training data')
  # else:
  #   print('Test data')

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device).squeeze(1)
      y = y.to(device)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f'{float(num_correct)/float(num_samples)*100:.2f}')

  model.train()

In [ ]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Training data
39.22
Test data
39.52
